### This is a project where I create a mouse pointer which works upon my hand gesture instructions

In [7]:
import cv2
import mediapipe as mp
import pyautogui
import util

In [8]:
screen_width , screen_height = pyautogui.size()

mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode = False,
    model_complexity = 1,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7,
    max_num_hands = 1
)



In [9]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    
    return None

In [10]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x*screen_width)
        y = int(index_finger_tip.y*screen_height)
        pyautogui.moveTo(x,y)

In [11]:
def detect_gestures(frame,landmark_list,processed):
    if len(landmark_list)>=21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.get_distance([landmark_list[4],landmark_list[5]])

        if thumb_index_dist < 50 and util.get_angle(landmark_list[5],landmark_list[6],landmark_list[8])>90:
            move_mouse(index_finger_tip)

In [12]:
def main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils

    try:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break
            frame = cv2.flip(frame,1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            processed = hands.process(frameRGB)
            landmarks_list = []
            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landmarks,mpHands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x,lm.y))

            detect_gestures(frame,landmarks_list,processed)


            cv2.imshow('Frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()



W0000 00:00:1740522267.282622   24683 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740522267.335886   24683 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x38bc3600) is not the object's thread (0x39107a90).
Cannot move to target thread (0x38bc3600)

QObject::moveToThread: Current thread (0x38bc3600) is not the object's thread (0x39107a90).
Cannot move to target thread (0x38bc3600)

QObject::moveToThread: Current thread (0x38bc3600) is not the object's thread (0x39107a90).
Cannot move to target thread (0x38bc3600)

QObject::moveToThread: Current thread (0x38bc3600) is not the object's thread (0x39107a90).
Cannot move to target thread (0x38bc3600)

QObject::moveToThread: Current thread (0x38bc3600) is not the object's thread (0x39107a90).
Cann

KeyboardInterrupt: 